# Train GAT models for Cora, Citeseer and Pubmed datasets

In [1]:
import torch
import torch.nn.functional as F
from models import GAT_L2_intervention

def train(model, optimizer, data, epochs):
    model.train()
    criterion = torch.nn.CrossEntropyLoss()

    for _ in range(epochs):
        optimizer.zero_grad()
        out = model(data.x, data.edge_index)
        loss = criterion(out[data.train_mask], data.y[data.train_mask])
        acc = (out[data.train_mask].argmax(dim=1) == data.y[data.train_mask]).sum().item() / data.train_mask.sum().item()
        loss.backward()
        optimizer.step()

    return model, loss, acc

@torch.no_grad()
def test(model, data):
    model.eval()
    criterion = torch.nn.CrossEntropyLoss()
    x, edge_index, y = data.x, data.edge_index, data.y
    out = model(x, edge_index)
    loss = criterion(out[data.test_mask], y[data.test_mask])
    acc = (out[data.test_mask].argmax(dim=1) == y[data.test_mask]).sum().item() / data.test_mask.sum().item()
    return loss, acc

In [2]:
def train_model_weight_decay(data, weight_decay):
    out_channels = data.y.max().item() + 1
    model = GAT_L2_intervention(in_channels=data.num_node_features, 
                                hidden_channels=64, 
                                out_channels=out_channels, 
                                heads=1)
    # Define the number of epochs
    epochs = 60
    # Define the learning rate
    lr = 0.001
    # Prepare the optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    # Train the models
    model, loss, acc = train(model=model, data=data, optimizer=optimizer, epochs=epochs)
    # Test the models
    test_loss, test_acc = test(model=model, data=data)
    # Print the results
    print(f"Model: GAT_Cora_2L1H_weight_decay_{weight_decay}, Loss: {loss:.4f}, Train Accuracy: {acc:.4f}, Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}")
    torch.save(model, f'/workspace/GAT_Cora_2L1H_weight_decay_{weight_decay}.pt')

def train_model_dropout(data, dropout):
    out_channels = data.y.max().item() + 1
    model = GAT_L2_intervention(in_channels=data.num_node_features, 
                                hidden_channels=64, 
                                out_channels=out_channels, 
                                heads=1,
                                dropout=dropout)
    # Define the number of epochs
    epochs = 60
    # Define the learning rate
    lr = 0.001
    # Prepare the optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=0)
    # Train the models
    model, loss, acc = train(model=model, data=data, optimizer=optimizer, epochs=epochs)
    # Test the models
    test_loss, test_acc = test(model=model, data=data)
    # Print the results
    print(f"Model: GAT_Cora_2L1H_dropout_{dropout}, Loss: {loss:.4f}, Train Accuracy: {acc:.4f}, Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}")
    torch.save(model, f'/workspace/GAT_Cora_2L1H_dropout_{dropout}.pt')

def train_model_general_hyperparameters(data, hidden_channels):
    out_channels = data.y.max().item() + 1
    model = GAT_L2_intervention(in_channels=data.num_node_features, 
                                hidden_channels=hidden_channels, 
                                out_channels=out_channels, 
                                heads=1)
    # Define the number of epochs
    epochs = 60
    # Define the learning rate
    lr = 0.001
    # Prepare the optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=0)
    # Train the models
    model, loss, acc = train(model=model, data=data, optimizer=optimizer, epochs=epochs)
    # Test the models
    test_loss, test_acc = test(model=model, data=data)
    # Print the results
    print(f"Model: GAT_Cora_2L1H_hidden_channels_{hidden_channels}, Loss: {loss:.4f}, Train Accuracy: {acc:.4f}, Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}")
    torch.save(model, f'/workspace/GAT_Cora_2L1H_hidden_channels_{hidden_channels}.pt')

## 1. Load Cora

In [3]:
import torch

data = torch.load('/workspace/Cora.pt', map_location="cpu")

## 2. Configurations

### 1. Regularizations

Weight decay: 0, 0.0001, 0.0005, 0.001, 0.005, 0.01

Dropout: 0, 0.2, 0.4, 0.6, 0.8

In [9]:
# for weight_decay in [0, 0.0001, 0.0005, 0.001, 0.005, 0.01]:
#     train_model_weight_decay(data=data, weight_decay=weight_decay)

Model: GAT_Cora_2L1H_weight_decay_0, Loss: 1.7281, Train Accuracy: 0.8986, Test Loss: 1.7639, Test Accuracy: 0.8162
Model: GAT_Cora_2L1H_weight_decay_0.0001, Loss: 1.7402, Train Accuracy: 0.9000, Test Loss: 1.7683, Test Accuracy: 0.8386
Model: GAT_Cora_2L1H_weight_decay_0.0005, Loss: 1.7782, Train Accuracy: 0.8886, Test Loss: 1.8053, Test Accuracy: 0.8352
Model: GAT_Cora_2L1H_weight_decay_0.001, Loss: 1.8046, Train Accuracy: 0.8857, Test Loss: 1.8224, Test Accuracy: 0.7973
Model: GAT_Cora_2L1H_weight_decay_0.005, Loss: 1.9151, Train Accuracy: 0.8529, Test Loss: 1.9176, Test Accuracy: 0.8068
Model: GAT_Cora_2L1H_weight_decay_0.01, Loss: 1.9293, Train Accuracy: 0.8057, Test Loss: 1.9315, Test Accuracy: 0.7764


In [10]:
# for dropout in [0, 0.2, 0.4, 0.6, 0.8]:
#     train_model_dropout(data=data, dropout=dropout)

Model: GAT_Cora_2L1H_dropout_0, Loss: 1.7304, Train Accuracy: 0.9057, Test Loss: 1.7588, Test Accuracy: 0.8332
Model: GAT_Cora_2L1H_dropout_0.2, Loss: 1.7311, Train Accuracy: 0.8986, Test Loss: 1.7573, Test Accuracy: 0.8401
Model: GAT_Cora_2L1H_dropout_0.4, Loss: 1.7380, Train Accuracy: 0.8829, Test Loss: 1.7713, Test Accuracy: 0.8267
Model: GAT_Cora_2L1H_dropout_0.6, Loss: 1.7505, Train Accuracy: 0.8700, Test Loss: 1.7794, Test Accuracy: 0.8262
Model: GAT_Cora_2L1H_dropout_0.8, Loss: 1.7645, Train Accuracy: 0.7886, Test Loss: 1.7950, Test Accuracy: 0.8237


In [4]:
# for hidden_channels in [32, 64, 128, 256]:
#     train_model_general_hyperparameters(data=data, hidden_channels=hidden_channels)

Model: GAT_Cora_2L1H_hidden_channels_32, Loss: 1.8122, Train Accuracy: 0.8886, Test Loss: 1.8312, Test Accuracy: 0.8391
Model: GAT_Cora_2L1H_hidden_channels_64, Loss: 1.7340, Train Accuracy: 0.9043, Test Loss: 1.7695, Test Accuracy: 0.8386
Model: GAT_Cora_2L1H_hidden_channels_128, Loss: 1.5905, Train Accuracy: 0.9086, Test Loss: 1.6440, Test Accuracy: 0.8471
Model: GAT_Cora_2L1H_hidden_channels_256, Loss: 1.3728, Train Accuracy: 0.8971, Test Loss: 1.4469, Test Accuracy: 0.8376
